In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# import mne
import numpy as np
# from copy import deepcopy
# import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
plot_params = dict(surface='white', hemi='both', verbose=0)

## Create Forward model
First we create a template forward model which comes with the esinet package

In [2]:
info = get_info(sfreq=100, kind="biosemi64")
fwd = create_forward_model(sampling="ico3", info=info)
_, pos = util.unpack_fwd(fwd)[1:3]
leadfield = fwd["sol"]["data"]
n_chans, n_dipoles = leadfield.shape

import mne
from scipy.sparse.csgraph import laplacian

adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
laplace_operator = abs(laplacian(adjacency))

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.7s remaining:    2.9s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.1s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


-- number of adjacent vertices : 1284


# Defs

In [3]:
def prep_data(X, y):
    X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
    y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)
    return X, y

# Simulate

In [217]:
settings = dict(duration_of_trial=0.25, extents=(1,40), number_of_sources=(1,15), target_snr=1e99)
sim = Simulation(fwd, info, settings=settings).simulate(n_samples=2000)
X = np.stack([eeg.average().data for eeg in sim.eeg_data], axis=0)
y = np.stack([source.data for source in sim.source_data], axis=0)
X, y = prep_data(X, y)
X = np.swapaxes(X, 1, 2)
y = np.swapaxes(y, 1, 2)

Simulating data based on sparse patches.


100%|██████████| 2000/2000 [00:26<00:00, 75.05it/s]


# Model

## Create

In [226]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer, Input, Lambda, LayerNormalization, GRU, multiply
from tensorflow.keras.layers import Activation, Dropout, ActivityRegularization, TimeDistributed, Reshape, Permute, GaussianNoise, add
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
# import tensorflow_probability as tfp
from tensorflow.keras.regularizers import l1, l2, l1_l2
K.set_image_data_format('channels_last')

# def data_loss(leadfield):
#     leadfield_ = tf.cast(leadfield, dtype=tf.float32)
#     def batch_data_loss(y_true, y_est):
#         def d_loss(y_true, y_est):
#             y_est = tf.transpose(tf.matmul(leadfield_, tf.transpose(y_est)))
#             # print("y_true ", y_true)
#             # print("y_est ", y_est)
            
#             # return K.mean(K.square(y_est - y_true))
#             return tf.keras.losses.CosineSimilarity(name="Data_Cosine_Loss")(y_est, y_true)
        
        

#         batched_losses = tf.map_fn(lambda x:
#             d_loss(x[0], x[1]), 
#             (y_true, y_est), dtype=tf.float32)
#         return K.mean(batched_losses)


#     return batch_data_loss

def data_loss(leadfield, lam_0=0.1):
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    def batch_data_loss(y_true, y_est):
        def d_loss(y_true, y_est):
            y_true_eeg = tf.transpose(tf.matmul(leadfield_, tf.transpose(y_true)))
            y_est_eeg = tf.transpose(tf.matmul(leadfield_, tf.transpose(y_est)))
            # print("y_true ", y_true)
            # print("y_est ", y_est)
            
            # return K.mean(K.square(y_est - y_true))
            error_source = tf.keras.losses.CosineSimilarity(name="Data_Cosine_Loss")(y_est, y_true)
            error_eeg = tf.keras.losses.CosineSimilarity(name="Data_Cosine_Loss")(y_est_eeg, y_true_eeg)
            return error_source*lam_0 + error_eeg
        
        

        batched_losses = tf.map_fn(lambda x:
            d_loss(x[0], x[1]), 
            (y_true, y_est), dtype=tf.float32)
        return K.mean(batched_losses)


    return batch_data_loss


def consistency(x):
    return K.mean(K.std(K.abs(x), axis=1))

# def consistency(source):
#     def c_loss(x):
#         matrix = compute_cosine_distances(K.abs(x), K.abs(x))
#         return K.mean(matrix)


#     batched_losses = tf.map_fn(lambda x:
#             c_loss(x), 
#             source, dtype=tf.float32)
#     return K.mean(batched_losses)

def c_loss(x):
        matrix = compute_cosine_distances(K.abs(x), K.abs(x))
        return K.mean(matrix)


def compute_cosine_distances(a, b):
    # x shape is n_a * dim
    # y shape is n_b * dim
    # results shape is n_a * n_b

    normalize_a = tf.nn.l2_normalize(a,1)        
    normalize_b = tf.nn.l2_normalize(b,1)
    distance = 1 - tf.matmul(normalize_a, normalize_b, transpose_b=True)
    return distance


def l1_sparsity(x):
    return K.mean(K.abs(x)) 

def get_model(name="Model", n_dense_layers=2, hidden_units=200, learning_rate=0.001, lam_0=0.1, lam_1=1, lam_2=0.001, add_consistency=True):
    input_shape = (None, n_chans)
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    inputs = tf.keras.Input(shape=input_shape, name=name)
    fc = TimeDistributed(Dense(hidden_units, activation="linear", name="FC1"))(inputs)

    gru = GRU(64, return_sequences=True, name='GRU_Discriminator')(inputs)
    source_time = TimeDistributed(Dense(n_dipoles, activation="sigmoid", name="Mask"))(gru)

    source = TimeDistributed(Dense(n_dipoles, activation="linear", name="Output_Final"))(fc)

    out = multiply([source_time, source])
    # print(source)
    # out = Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans))(source)
    # # out = TimeDistributed(Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans)))(source)
    
    
    model = tf.keras.Model(inputs=inputs, outputs=out, name='CS_Net')
    
    # Data Loss
    # L1 Loss
    model.add_loss(l1_sparsity(source)*lam_1)
    
    # Data consistency loss
    if add_consistency:
        model.add_loss(c_loss(source)*lam_2)
    
    model.compile(loss=data_loss(leadfield, lam_0=lam_0), optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    # model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    
    return model



## Train

In [227]:
lams = [0.001, ]
models = []
for lam in lams:
    model = get_model(lam_2=lam, name=f"Lam {lam}")
    model.fit(X, y, epochs=20, batch_size=8, validation_split=0.1)
    models.append(model)


Epoch 1/20
225/225 [==============================] - 12s 46ms/step - loss: -0.8990 - val_loss: -0.9696
Epoch 2/20
225/225 [==============================] - 9s 42ms/step - loss: -0.9872 - val_loss: -1.0030
Epoch 3/20
225/225 [==============================] - 10s 42ms/step - loss: -1.0026 - val_loss: -0.9996
Epoch 4/20
225/225 [==============================] - 9s 41ms/step - loss: -1.0107 - val_loss: -1.0122
Epoch 5/20
225/225 [==============================] - 9s 40ms/step - loss: -1.0188 - val_loss: -1.0219
Epoch 6/20
225/225 [==============================] - 9s 42ms/step - loss: -1.0212 - val_loss: -1.0217
Epoch 7/20
225/225 [==============================] - 10s 45ms/step - loss: -1.0249 - val_loss: -1.0229
Epoch 8/20
225/225 [==============================] - 10s 45ms/step - loss: -1.0297 - val_loss: -1.0166
Epoch 9/20
225/225 [==============================] - 10s 45ms/step - loss: -1.0310 - val_loss: -1.0287
Epoch 10/20
225/225 [==============================] - 10s 45ms/step

## Evaluate

In [197]:
from esinet.evaluate import eval_auc
from scipy.stats import pearsonr
sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=2)
idx = 0
stc = sim_test.source_data[idx].copy()
stc.plot(**plot_params)

for mod in models:
    src_hat = mod.predict(X)[idx]
    stc_hat = stc.copy()
    stc_hat.data = src_hat.T
    stc_hat.plot(**plot_params)
    auc = np.mean([np.mean(eval_auc(y_true_sample, y_pred_sample, pos)) for y_true_sample, y_pred_sample in zip(stc.data.T, stc_hat.data.T)])
    corrs = np.mean([np.mean(pearsonr(y_true_sample, y_pred_sample)[0]) for y_true_sample, y_pred_sample in zip(stc.data.T, stc_hat.data.T)])
    nmses = np.mean([np.mean((y_true_sample - y_pred_sample)**2) for y_true_sample, y_pred_sample in zip(stc.data.T, stc_hat.data.T)])
    
    print("AUC: ", auc, " Corrs: ", corrs, " nMSE: ", nmses)

# Evaluate New

In [237]:
from esinet.evaluate import eval_auc
from scipy.stats import pearsonr
sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=2)

X_test = np.stack([eeg.average().data for eeg in sim_test.eeg_data], axis=0)
y_test = np.stack([source.data for source in sim_test.source_data], axis=0)
X_test, y_test = prep_data(X_test, y_test)
X_test = np.swapaxes(X_test, 1, 2)
y_test = np.swapaxes(y_test, 1, 2)


idx = 0
stc = sim_test.source_data[idx].copy()
stc.plot(**plot_params)

for model in models:
    src_hat = model.predict(X_test)[idx]
    stc_hat = stc.copy()
    stc_hat.data = src_hat.T
    stc_hat.plot(**plot_params)
    auc = np.mean([np.mean(eval_auc(y_true_sample, y_pred_sample, pos)) for y_true_sample, y_pred_sample in zip(stc.data.T, stc_hat.data.T)])
    corrs = np.mean([np.mean(pearsonr(y_true_sample, y_pred_sample)[0]) for y_true_sample, y_pred_sample in zip(stc.data.T, stc_hat.data.T)])
    nmses = np.mean([np.mean((y_true_sample - y_pred_sample)**2) for y_true_sample, y_pred_sample in zip(stc.data.T, stc_hat.data.T)])
    
    print(model.name, " AUC: ", auc, " Corrs: ", corrs, " nMSE: ", nmses)

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 77.85it/s]


CS_Net  AUC:  0.8531631145033631  Corrs:  0.47838636132969276  nMSE:  0.00013167695252763248


Using control points [0.05064677 0.06514393 0.13835773]
Using control points [0.05064677 0.06514393 0.13835773]
